In [12]:
import numpy as np
import scipy.stats as stats
import statsmodels


def generate_data(n, p):
    x = np.random.binomial(n, p)
    return x


def compute_inverse_cdf_of_standard_normal(prob=0.975):
    inverse_cdf = stats.norm.ppf(prob)
    return inverse_cdf


def normal_confidence_interval(n, x, confidence_level=0.95):
    p_hat = x / n
    alpha = 1 - confidence_level
    z = compute_inverse_cdf_of_standard_normal(prob=1-(alpha/2)) 
    
    center = p_hat
    multiplier = z
    width = np.sqrt(p_hat*(1-p_hat)/n) 
    
    lo = center - multiplier * width 
    hi = center + multiplier * width
    return lo, hi


def wilson(n, x, confidence_level=0.95):
    p_hat = x / n
    alpha = 1 - confidence_level
    z = compute_inverse_cdf_of_standard_normal(prob=1-(alpha/2)) 
    
    center = (p_hat + (z**2/(2*n))) / (1 + (z**2/(n))) 
    multiplier = z / (1 + (z**2/n)) 
    width = np.sqrt((p_hat*(1-p_hat)/n) + (z**2/(4*(n**2)))) 
    
    lo = center - multiplier * width 
    hi = center + multiplier * width
    return lo, hi


def wilson_with_continuity_correction(n, x, confidence_level=0.95):
    p_hat = x / n
    alpha = 1 - confidence_level
    z = compute_inverse_cdf_of_standard_normal(prob=1-(alpha/2)) 
    
    center = (2*n*p_hat + z**2) / (2*(n+z**2)) 
    width_lower = (z*np.sqrt(z**2-(1/n)+4*n*p_hat*(1-p_hat)+(4*p_hat-2))+1) / (2*(n+z**2)) 
    width_upper = (z*np.sqrt(z**2-(1/n)+4*n*p_hat*(1-p_hat)-(4*p_hat-2))+1) / (2*(n+z**2))
    
    lo = center - width_lower 
    hi = center + width_upper
    lo = np.maximum(0, lo)
    hi = np.minimum(1, hi)
    return lo, hi


def jeffreys(n, x, confidence_level=0.95):
    # http://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html
    # https://github.com/statsmodels/statsmodels/issues/3167
    alpha = 1 - confidence_level
    lo, hi = statsmodels.stats.proportion.proportion_confint(count=x, nobs=n, alpha=alpha, method='jeffrey')
    return lo, hi


def binom_test(n, x, confidence_level=0.95):
    # http://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html
    alpha = 1 - confidence_level
    lo, hi = statsmodels.stats.proportion.proportion_confint(count=x, nobs=n, alpha=alpha, method='binom_test')
    return lo, hi


def clopper_pearson_beta(n, x, confidence_level=0.95):
    # https://gist.github.com/DavidWalz/8538435
    """
    http://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval
    alpha confidence intervals for a binomial distribution of k expected successes on n trials
    Clopper Pearson intervals are a conservative estimate.
    """
    alpha = 1 - confidence_level
    lo = stats.beta.ppf(alpha/2, x, n-x+1)
    hi = stats.beta.ppf(1 - alpha/2, x+1, n-x)
    return lo, hi


def agresti_coull(n, x, confidence_level=0.95):
    alpha = 1 - confidence_level
    z = compute_inverse_cdf_of_standard_normal(prob=1-(alpha/2)) 
    n_tilde = n + z**2
    x_tilde = x + (z**2/2)
    p_tilde = x_tilde / n_tilde

    center = p_tilde
    multiplier = z
    width = np.sqrt(p_tilde*(1-p_tilde)/n_tilde) 
    
    lo = center - multiplier * width 
    hi = center + multiplier * width
    return lo, hi


if __name__ == '__main__':
    n = 1000
    p = 0.3
    confidence_level = 0.95
    
    x = generate_data(n, p)
    print(x)
    
    lo, hi = normal_confidence_interval(n, x, confidence_level)
    print("Normal:          ", [lo, hi])
    #print(statsmodels.stats.proportion.proportion_confint(x, n, alpha=1-confidence_level, method='normal'))
    
    lo, hi = wilson(n, x, confidence_level)
    print("Wilson:          ", [lo, hi])
    #print(statsmodels.stats.proportion.proportion_confint(x, n, alpha=1-confidence_level, method='wilson'))
    
    lo, hi = wilson_with_continuity_correction(n, x, confidence_level)
    print("Wilson_c_corr:   ", [lo, hi])
    
    lo, hi = jeffreys(n, x, confidence_level)
    print("Jeffreys:        ", [lo, hi])
    #print(statsmodels.stats.proportion.proportion_confint(x, n, alpha=1-confidence_level, method='jeffreys'))
    
    lo, hi = binom_test(n, x, confidence_level)
    print("binom_test:      ", [lo, hi])
    #print(statsmodels.stats.proportion.proportion_confint(x, n, alpha=1-confidence_level, method='binom_test'))
    
    lo, hi = clopper_pearson_beta(n, x, confidence_level)
    print("Clopper-Pearson: ", [lo, hi])
    #print(statsmodels.stats.proportion.proportion_confint(x, n, alpha=1-confidence_level, method='beta'))
    
    lo, hi = agresti_coull(n, x, confidence_level)
    print("Agresti-Coull:   ", [lo, hi])
    #print(statsmodels.stats.proportion.proportion_confint(x, n, alpha=1-confidence_level, method='agresti_coull'))

303
Normal:           [0.2745169957728962, 0.3314830042271038]
Wilson:           [0.27531542432193984, 0.3321923185213688]
Wilson_c_corr:    [0.274830561250732, 0.33270358904930564]
Jeffreys:         [0.2751172736485245, 0.33202120247296496]
binom_test:       [0.2749300307932193, 0.3324485743829511]
Clopper-Pearson:  [0.2746319878756163, 0.3325329876045753]
Agresti-Coull:    [0.27530547215680734, 0.33220227068650143]


# Reference

[wiki](https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval)

[statsmodels](http://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportion_confint.html)